# Flight Delay Prediction Part 4 - Delay Prediction

We will be predicting the arrival delay of the flights.

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Load all requirements
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
df = pd.read_csv("RESULT.csv")

In [ ]:
# Train only on flights that have an arrival delay
df = df[df["ARR_DEL15"]==1]
# Labels must be predicted
labels = np.asarray(df["ARR_DELAY_NEW"])
# Drop unwanted columns from the feature DataFrame
df_select = df.drop(["OP_CARRIER_FL_NUM","DEP_DELAY","DEP_DEL15","DEP_TIME","ARR_DELAY","ARR_DELAY_NEW","ARR_DEL15","ARR_TIME","date","airport","index","FL_DATE",'hour'],axis=1)
df_select = df_select.reset_index(drop=True)
df_select.drop(['OP_UNIQUE_CARRIER'],axis=1, inplace=True)
df_select.drop(['ORIGIN'],axis=1, inplace=True)
# Sample one-hot encoding statement:
# df_select = pd.concat([df_select,pd.get_dummies(df_select['DEST'], prefix='DEST')],axis=1)
df_select.drop(['DEST'],axis=1, inplace=True)
df_select.drop(['winddir16Point'],axis=1, inplace=True)
# Features are the input based on which we must predict the delay value
features = np.asarray(df_select)

In [ ]:
df_select.keys()

### Gradient Boosting Regressor

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.20, random_state=42)
from sklearn.ensemble import GradientBoostingRegressor
lr = GradientBoostingRegressor()
lr.fit(features_train,labels_train)
pred = lr.predict(features_test)
acc_test = lr.score(features_test,labels_test)
print ("Test Accuracy:", acc_test)

### Extra Trees Regressor

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor
df.dropna(inplace=True)
# Train only on delayed flight data
df1 = df[df['ARR_DEL15']==1]
# Training on 2016 data
train = df1[df1['YEAR']==2016]
# Testing on 2017 data
test = df1[df1['YEAR']==2017]
# X is feature set, drop unwanted columns - for training
X = train.drop(["DEP_DEL15","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DEL15","ARR_DELAY","ARR_DELAY_NEW","DEST","ORIGIN","index",'FL_DATE',"OP_UNIQUE_CARRIER",'hour','date','airport','winddir16Point'],axis=1)
# y is the label to predict - training
y = train['ARR_DELAY_NEW']
# Testing X and y arrays
test_x = test.drop(["DEP_DEL15","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DEL15","ARR_DELAY","ARR_DELAY_NEW","DEST","ORIGIN","index",'FL_DATE',"OP_UNIQUE_CARRIER",'hour','date','airport','winddir16Point'],axis=1)
test_y = test['ARR_DELAY_NEW']
# Regressor
etr = ExtraTreesRegressor(n_estimators=200,n_jobs=-1,verbose=True)
etr.fit(X,y)
val = etr.predict(test_x)
# Errors
print("MAE:\t",mean_absolute_error(test_y,val))
print("RMSE:\t",np.sqrt(mean_squared_error(test_y,val)))
print("R2:\t",r2_score(test_y,val))

In [ ]:
X.keys()

In [ ]:
etr.feature_importances_